In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(0 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [3]:
train_user = pd.read_csv('../input/训练数据-ccf_first_round_user_shop_behavior.csv')
test = pd.read_csv('../input/AB榜测试集-evaluation_public.csv')

train_user.shape
train_user.head()
test.shape
test.head()

(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [4]:
mall_ids = set(test.mall_id)
userShop = train_user.shop_id
userShop.head()

del train_user
del test

0    s_2871718
1    s_2871718
2     s_181637
3     s_609470
4    s_3816766
Name: shop_id, dtype: object

In [5]:
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

In [9]:
mall_wifi_dateframe('m_622', 'test_matrix')

,full_order,b_3758640,b_3155503,b_56331008,b_33489263,b_18655345,b_9091070,b_6724355,b_49662085,b_19188542,...,b_18304278,b_41824143,b_55555065,b_23622024,b_19054820,b_20952577,b_39241838,b_40230368,b_53654217,b_56549844
0,1138172,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1138212,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1138238,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1138239,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1138240,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1138241,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1138242,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1138243,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1138270,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1138283,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import os

finish_mall = os.listdir('./output/test_user_neg10/')
for i in range(len(finish_mall)):
    finish_mall[i] = finish_mall[i][:-2]
    
finish_mall

['m_5892', 'm_6803', 'm_5810', 'm_3425', 'm_3112', 'm_3054']

In [10]:
len(finish_mall)

6

In [ ]:
for mall_id in tqdm(mall_ids):
    if mall_id not in finish_mall:
        train = mall_wifi_dateframe(mall_id, 'train_matrix')
        test = mall_wifi_dateframe(mall_id, 'test_matrix')
        l_train = len(train)
        l_test = len(test)
        full_order = train.full_order
        test_full_order = test.full_order
        mall = pd.concat([train, test])
        del train
        del test
        mall.reset_index(drop=True, inplace=True)
        temp = pd.DataFrame(1-pairwise_distances(mall.drop(['full_order'], axis=1).as_matrix(), metric="cosine"))
        del mall
        user_frame = pd.DataFrame()
    
        for i in range(l_test):
            ii = l_train + i
            k = 9
            p = 0
            q = 0
            user = []
            shop_list=[]
            user_raw = test_full_order[i] # test第一行要加一个偏移
            user.append(user_raw)
            index = temp.iloc[ii].sort_values(ascending=False).index
            while p < k:
                if index[q] >= l_train:
                    q = q + 1
                else:
                    if (userShop[full_order[index[q]]] not in shop_list) & (full_order[index[q]] != user_raw):
                        user.append(full_order[index[q]])
                        shop_list.append(userShop[full_order[index[q]]])
                        q = q + 1
                        p = p + 1
                    else:
                        q = q + 1
            user_frame = pd.concat([user_frame, pd.DataFrame(user)], axis=1)
        
        print(mall_id)
        finish_mall.append(mall_id)
        pickle.dump(user_frame, open('./output/test_user_neg10/' + str(mall_id) + '.p', 'wb'), protocol=4)


  0%|          | 0/97 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/miniconda2/envs/py35/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  7%|▋         | 7/97 [11:52<2:32:36, 101.74s/it]

m_6337
